<h1>Double  Integration</h1>

<p>We load some packages and then begin:</p>

In [1]:
using Plots, SymPy, LinearAlgebra

<p>Let $f: R^2 \rightarrow R$ be a continuous, real-valued function of two variables. If $f(x,y) \geq 0$ over a region $D$, then the volume under $f$ above the $x-y$ plan is given by an integral</p>


$$
\int_D f(x,y) dA
$$


<p>More generally, if $f(x,y)$ is possibly negative, then the above is interpreted as the signed volume. As with one-dimensional integrals, this insight can lead to methods to numerically compute the integral above. For now we focus on applying the fundamental theorem of calculus to perform the integration. For this, the double integral must be turned into a sequence of single integrals.</p>

<h2>Fubini's thereom, iterated integrals</h2>

<p><a href="http://en.wikipedia.org/wiki/Fubini&#37;27s_theorem">Fubini's theorem</a> gives conditions on when it is possible to turn a double (or higher dimensional integral) into a sequence of iterated integrals. Fubini's theorem will apply if the region $D$ is the rectangle $[a,b] \times [c,d]$ and the function $f$ is continuous, in which case we have</p>


$$
\int_D f(x,y) dA = \int_a^b [\int_c^d f(x,y) dy] dx = \int_c^d [\int_a^b f(x,y) dx] dy.
$$


<p>The value on the right can be computed by hand or with <code>SymPy</code>'s <code>integrate</code> function within <code>julia</code>.</p>

<p>For example, consider the area under the plane $f(x,y) = 1 - (x + y)/2$ over the region $[0,1] \times [0,1]$. Fubini's theorem says this is simply</p>


$$
\int_0^1 [\int_0^1 (1 - (x+y)/2) dy] dx.
$$


<p>This can be computed as:</p>

In [1]:
using SymPy
@vars x y z real=true

integrate(1 - (x + y)/2, (y, 0, 1), (x, 0, 1))

<p>The limits of integration are specified with the innermost first and outermost last. They are specified with a triple. So the math notation</p>


$$
\int_a^b (...) dx
$$


<p>Becomes <code>integrate&#40; ..., &#40;x, a, b&#41;&#41;</code> for whatever <code>...</code> means.</p>

<h2>Limits of integration</h2>

<p>More generally, the region may be more complicated than a rectangle. If $D$ is "simple", then one can say either $a \leq x \leq b$ and $g_1(x) \leq y \leq g_2(x)$ (or vice-versa in $x$ and $y$) for some functions $g_1$ and $g_2$, then the double integral over $D$ can be iterated to:</p>


$$
\int_a^b \int_{g_1(x)}^{g_2(x)} f(x,y) dy dx.
$$


<p>For example, suppose $D$ is the region in the $x-y$ plane described by $0 \leq x \leq 1$ and $0 \leq y \leq x^2$, then the integral of $(x+y)^2$ over $D$ can be computed by</p>

In [1]:
integrate((x+y)^2, (y, 0, x^2), (x,0,1))

<h2>polar coordinates</h2>

<p>If it is more convenient to express the domain $D$ in <em>polar coordinates</em>, then the double integral of $f$ over $D$ can be re-expressed as:</p>


$$
\int_D f(x,y) dA =  \int_\theta \int_r f(r\cos(\theta), r\sin(\theta)) \cdot r \cdot dr \cdot d\theta.
$$


<p>The $r \cdot dr \cdot d\theta$ matches $dA$ after the transformation, of which more is described later.</p>

<p>For example, consider the problem of integrating $f(x,y) = x^2 + y^2$ over the unit circle. Using rectangular coordinates we have an error:</p>

In [1]:
# integrate(x^2 + y^2, (y, -sqrt(1-x^2), sqrt(1-x^2)), (x, -1, 1))  ## uncomment to see error

<p>We can see where the error comes from by doing it one at a time:</p>

In [1]:
@vars x y real=true
ex = integrate(x^2 + y^2, y)
ex1 = subs(ex, y, sqrt(1-x^2)) - subs(ex, y, -sqrt(1-x^2))

<p>Trying <code>integrate&#40;ex1&#41;</code> should work – and does – but for some reason it also fails at times. Here we break this up into two pieces and integrate each:</p>

In [1]:
exa = 2 * x^2 * sqrt(1 - x^2)
exb = (2//3) * sqrt(1 - x^2)^3

<p>The second integral gives:</p>

In [1]:
outb = integrate(exb, x)

<p>(Though there can be errors some times when this is computed.)</p>

<p>The first integral has a condition:</p>

In [1]:
out = integrate(exa, x)

<p>We can fish out the value for $|x| < 1$, but it isn't pretty:</p>

In [1]:
outa = 2*collect((out/2).x.args[2].x)[1]

<p>With this we can see the issue – evaluating at 1 or -1 gives an <code>NaN</code>. We could take a limit – but this doesn't work – rather we just put in a small value of $h$ and see:</p>

In [1]:
h = 0.0001
subs(outa + outb, x, 1- h) - subs(outa + outb, x, -1 + h)

<p>An answer that looks like $\pi/2$.</p>

<p>Okay, that was too much work and still not that satisfying! Let's see what happens if we change coordinate systems. The unit circle is described by $0 \leq r \leq 1$ and $0 \leq \theta \leq 2\pi$ – a simple region, and we have</p>

In [1]:
@vars r theta real=true
f(x,y) = x^2 + y^2
integrate(f(r*cos(theta), r*sin(theta)) * r, (theta, 0, 2PI), (r, 0, 1)) # note extra "* r"

<h2>Triple integrals</h2>

<p>Triple integrals are conceptually no more difficult than double integrals. Fubini's theorem is used to iterate the integrals, and if a region can be written iteratively, then the process is similar.</p>

<p>For example, compute $\int\int\int_D z dV$ where $D$ is the region between the planes $z=x+y$, $z=3x+5y$ lying over the rectangle $[0,3] \times [0,2]$. This becomes:</p>

In [1]:
@vars x y z
integrate(z, (z, x+y, 3x+5y), (y, 0, 2), (x, 0,3))

<h2>cylindrical, spherical</h2>

<p>As polar coordinates make certain double integrals easier, triple integrals can be easier to compute using either cylindrical coordinate systems or spherical. This relates them:</p>


$$
\int_D f(x,y,z) dV = \int_\theta \int_r \int_z f(r\cos(\theta), r\sin(\theta)z) r dz dr d\theta
= \int_\theta \int_\phi \int_\rho f(\rho\cos\theta\sin\phi, \rho\sin\theta\sin\phi, \rho\cos\phi) \rho^2 \sin\phi d\rho d\phi d\theta.
$$


<p>For example, computing the integral of $z$ over the region $D$ described by it fitting in the cylinder $x^2 + y^2\leq 4$ with $0 \leq z \leq y$ we have noting that the cylinder is a circle with radius 2:</p>

In [1]:
integrate(z * r, (z, 0, r*sin(theta)), (r, 0, 2), (theta, 0, PI))

<p>The subtlety above is the limit of integration on $\theta$ which is only $[0,\pi]$, as the condition that $0 \leq z \leq y$ only allows values when the plane $z=y$ is positive, or the upper half of the $x-y$ plane.</p>

<h2>Change of variables</h2>

<p>The integrands for polar, cylindrical, and spherical integrals look like $f \cdot G$ for some term $G$. (For example, $G=r$ for polar and cylindrical.) The $G$ comes from investigating the change in the "$dA$" or "$dV$" under the tranformation. What exactly $G$ is follows from the change of variable formula, written in two dimensions here:</p>


$$
\int_D f(x,y) dx dy = \int_D f(u,v) G(x,y;u,v) du dv.
$$


<p>The $G$ is the determinant of the Jacobian, $J$, that is the determinant of the matrix (in this case) $\partial x_i/ \partial u_j$.</p>

<ul>
<li><p>For example, suppose we have the transformation $x = r\sin(\theta)$ and $y=r\cos(\theta)$. Then $J$ is computed with:</p>
</li>
</ul>

In [1]:
@vars r theta real=true
xs = [r*cos(theta), r*sin(theta)]
J = [diff(x, u) for x in xs, u in (r, theta)]

<p>We help <code>SymPy</code> out by specifying the type of the output on the comprehension. We see <code>J</code> is a 2 by 2 matrix and its determinant can be taken with <code>det</code> and then simplified through:</p>

In [1]:
det(J) |> simplify

<ul>
<li><p>To see how spherical coordinates can fit into the change of variable formula, we have:</p>
</li>
</ul>

In [1]:
@vars rho phi theta real=true
xs = [rho*cos(theta)*sin(phi), rho*sin(theta)*sin(phi), rho*cos(phi)]
J = Sym[diff(x, u) for x in xs, u in (rho, phi, theta)]
det(J) |> simplify

<ul>
<li><p>Here is an application where we aren't just verifying an output. Let $D$ be the region between the lines $y=1$, $y=3$ and the parallel lines $x+2y=6$ and $x+2y=10$. Integrate $(x+3y)$.</p>
</li>
</ul>

<p>The change of variable $x =  u - 2v$, $y=v$ translates into</p>

In [1]:
@vars u v real=true
xs = x, y = u - 2v, v
x+2y

<p>So with these coordinate, $D$ becomes a rectangular region $1 \leq v \leq 3$ and $6 \leq u \leq 10$. We then have:</p>

In [1]:
J = Sym[diff(x, u) for x in xs, u in (u, v)]
integrate((x + 3y) * det(J), (v, 1, 3), (u, 6, 10))

<p>As an aside, this is an example of a <em>linear transformation</em> called a shear. From the linear algebra viewpoint, the transformation can be written in matrix notation as <code>xs &#61; J * us</code> (if <code>us</code> and <code>xs</code> were vectors and not tuples). That <code>J</code> is the same as the Jacobian is related to the linear nature of the transformation.</p>

<h2>Applications</h2>

<ul>
<li><p>Compute the area of the right half of the centroid $r^2 = \cos(2\theta)$.</p>
</li>
</ul>

<p>The key is $\theta$ goes from $-\pi/4$ to $\pi/4$ to trace out the region, giving:</p>

In [1]:
integrate(1*r, (r, 0, sqrt(cos(2theta))), (theta, -PI/4, PI/4))

<ul>
<li><p>Find the center of mass of the domain bounded by $y=1-x^2$ and the $x$ axis with density $\rho(x,y) = y$.</p>
</li>
</ul>

<p>The center of mass in the $x$ direction is given by $(\int_D x \cdot \rho \cdot DA)/M$ and similarly with a $y$.</p>

<p>We have:</p>

In [1]:
@vars x y real=true
rho = y
M  = integrate(rho, (y, 0, 1-x^2), (x, -1, 1))
Mx = integrate(x * rho, (y, 0, 1-x^2), (x, -1, 1))
My = integrate(y * rho, (y, 0, 1-x^2), (x, -1, 1))
Mx/M, My/M

(0, 4/7)

<p>That $M_x$ is $0$ is no surprise, as there is symmetry in both the region and the density to the left and right of the line $x=0$.</p>